In [1]:
import kotlinx.serialization.json.Json
import kotlinx.serialization.json.JsonArray
import kotlinx.serialization.json.JsonElement
import kotlinx.serialization.json.JsonNull
import kotlinx.serialization.json.JsonObject
import kotlinx.serialization.json.JsonPrimitive
import kotlinx.serialization.json.boolean
import kotlinx.serialization.json.booleanOrNull
import kotlinx.serialization.json.double
import kotlinx.serialization.json.doubleOrNull
import kotlinx.serialization.json.jsonArray
import org.mvel2.MVEL.eval
import org.mvel2.CompileException
import org.mvel2.templates.TemplateCompiler
import org.mvel2.templates.TemplateRuntime

Create sample JSON string with objects, arrays and primitive types (string, boolean, numbers)

In [2]:
val sample = """
{
  "orderId": "ORD-2025-09-15-001",
  "total": 149.99,
  "orderedAt": "2025-09-15T14:30:00Z",
  "customer": {
    "name": "Alice Smith",
    "age": 30,
    "vip": true
  },
  "items": [
    {
      "sku": "KB-2025-B",
      "desc": "Mechanical keyboard",
      "qty": 1,
      "unitPrice": 119.99
    },
    {
      "sku": "MS-2025-X",
      "desc": "Wireless mouse",
      "qty": 2,
      "unitPrice": 15.00
    }
  ]
}
""".trimIndent()

Parse JSON to JsonElement

In [3]:
val root = Json.parseToJsonElement(sample)

Create template text with MVEL syntax and custom fields like \\${field} \\${object.field} \@{foreach....}

In [4]:
val templateString = """
Order No: ${'$'}{orderId} ordered at ${'$'}{orderedAt}
Customer: ${'$'}{customer.name} (${'$'}{customer.age} years old) ${'$'}{customer.vip == true ? 'is VIP' : 'regular'}
Items:
@foreach{item : items}
  - ${'$'}{item.sku}: ${'$'}{item.desc} x${'$'}{item.qty} @ ${'$'}{item.unitPrice} = ${'$'}{item.qty * item.unitPrice}
@end{}
===================================================
TOTAL: ${'$'}{total}
""".trimIndent()

Helper function to map JsonElement to plain Kotlin types

In [5]:
fun JsonElement.toPlain(): Any? = when (this) {
    is JsonObject -> this.mapValues { it.value.toPlain() }
    is JsonArray -> this.map { it.toPlain() }
    is JsonPrimitive -> when {
        this.isString -> this.content
        this.booleanOrNull != null -> this.boolean
        this.doubleOrNull != null -> this.double
        else -> this.content
    }
    JsonNull -> null
}

val context = root.toPlain()
println(context)

{orderId=ORD-2025-09-15-001, total=149.99, orderedAt=2025-09-15T14:30:00Z, customer={name=Alice Smith, age=30.0, vip=true}, items=[{sku=KB-2025-B, desc=Mechanical keyboard, qty=1.0, unitPrice=119.99}, {sku=MS-2025-X, desc=Wireless mouse, qty=2.0, unitPrice=15.0}]}


Now, MVEL magic with two steps:
1. Compile template to context for evaluation
2. Execute template with context

In [6]:
val compiled = TemplateCompiler.compileTemplate(templateString)
val mergedString = TemplateRuntime.execute(compiled, context)

println(mergedString)

Order No: ORD-2025-09-15-001 ordered at 2025-09-15T14:30:00Z
Customer: Alice Smith (30.0 years old) is VIP
Items:

  - KB-2025-B: Mechanical keyboard x1.0 @ 119.99 = 119.99

  - MS-2025-X: Wireless mouse x2.0 @ 15.0 = 30.0

TOTAL: 149.99
